# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [42]:
#Your code here
import pandas as pd
import numpy as np
import scipy.stats as stats

df = pd.read_csv('homepage_actions.csv')
df.head()


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [43]:
#Your code here

df_stat = pd.crosstab(df.group, df.action)
df_stat['Not Clicked'] = df_stat['view'] - df_stat['click']
observ = df_stat[['click', 'Not Clicked']]

# define hypothesis
h0 = 'Experimental homepage was less effective than Control homepage'
h1 = 'Experimental homepage was more effective than Control Group'

# create a hypothesis testing fuction
def Hypothesis_testing(p_value, alpha):
    if p_value <= alpha:
        print(f'Reject null hypothesis as p value < {alpha}')
        print(h1)
    else:
        print(f'Fail to reject null hypothesis p value > {alpha}')
        print(h0) 

# conduct a chi-square test
import scipy.stats as stats
chi2, p_value, dof, expected = stats.chi2_contingency(observ)

print(f" Chi_stat : {chi2:.4f}")
print(f" p_value : {p_value:.4f}")
print(f" degrees of freedom : {dof}")
print(f" expected : {expected}\n")

Hypothesis_testing(p_value, 0.05)

 Chi_stat : 6.7129
 p_value : 0.0096
 degrees of freedom : 1
 expected : [[ 979.38053097 2352.61946903]
 [ 880.61946903 2115.38053097]]

Reject null hypothesis as p value < 0.05
Experimental homepage was more effective than Control Group


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [44]:
#Your code here
n_ctrl = len(df[df.group == 'control']['id'].unique())
n_exp = len(df[df.group == 'experiment']['id'].unique())

clicked_ctrl = len(df[(df.group == 'control') & (df.action == 'click')]['id'].unique())
clicked_exp = len(df[(df.group == 'experiment') & (df.action == 'click')]['id'].unique())
# print(f"Number of experiment observation: {n_ctrl}")
# print(f"Number of control observation: {n_exp}")

p_ctrl = clicked_ctrl/n_ctrl

expected_clicked_exp = p_ctrl * n_exp
expected_clicked_exp

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [45]:
#Your code here
var = expected_clicked_exp*(1-p_ctrl)
std = np.sqrt(var)
std

24.568547907005815

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [49]:
#Your code here
z = (clicked_exp - expected_clicked_exp)/std

p_value = 1-stats.norm.cdf(z)  # z > 0
p_value

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: Yes. The p value are different but both lead to the same conclusion of Rejecting the null hypothesis

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.